In [1]:
import neat
import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics import mean_squared_error, mean_absolute_error
from reservoirpy import datasets

import pureples as ples
from pureples.hyperneat.hyperneat import create_phenotype_network
from pureples.shared.substrate import Substrate


import reservoirpy as rpy
rpy.verbosity(0)

ModuleNotFoundError: No module named 'ples'

In [ ]:
# Define other required parameters for HyperNEAT experiment
population_size = 50
n_generations = 200
genome_length = 100  # This may not be directly applicable in HyperNEAT as in NEAT
n_timesteps = 2000

# Initialize your dataset for HyperNEAT
X = datasets.mackey_glass(n_timesteps=n_timesteps, sample_len=2000)
Y = np.roll(X, -1)

# Dataset split
train_end = int(len(X) * 0.7)
test_start = train_end + 1
X_train, Y_train = X[:train_end], Y[:train_end]
X_test, Y_test = X[test_start:], Y[test_start:]

# Ensure X and Y are correctly shaped for input into the neural network
X_train = X_train.reshape(-1, 1)
Y_train = Y_train.reshape(-1, 1)
X_test = X_test.reshape(-1, 1)
Y_test = Y_test.reshape(-1, 1)

In [ ]:
def eval_genomes_hyperneat(genomes, config):
    fitness_scores = []
    for genome_id, genome in genomes:
        # This function should be adapted to your implementation
        net = create_phenotype_network(genome, config)
        
        predictions = []
        for xi in X_train:  # Ensure xi is presented in the correct shape or form for your substrate
            output = net.activate(xi.flatten())  # Flattening might be needed depending on your input format
            predictions.append(output)
        
        predictions = np.squeeze(np.array(predictions))  # Ensure predictions are in the correct shape for comparison
        mse = mean_squared_error(Y_train, predictions)
        mae = mean_absolute_error(Y_train, predictions)
        rmse = np.sqrt(mse)
        
        fitness = 1 / (mse + 1.0)  # Example fitness function; can be customized
        genome.fitness = fitness
        fitness_scores.append(fitness)
    
    return fitness_scores

# Additional setup for HyperNEAT, including creating a substrate configuration
# and modifying the configuration loading as needed for HyperNEAT
config_path = 'config_hyperneat.txt'
config = neat.Config(neat.DefaultGenome, neat.DefaultReproduction, neat.DefaultSpeciesSet, neat.DefaultStagnation, config_path)

p = neat.Population(config)
p.add_reporter(neat.StdOutReporter(True))
stats = neat.StatisticsReporter()
p.add_reporter(stats)
p.add_reporter(neat.Checkpointer(5))

winner = p.run(eval_genomes_hyperneat, 50)

print('\nBest genome:\n{!s}'.format(winner))

# Plotting the performance graph
generation = range(len(stats.get_fitness_mean()))
plt.figure(figsize=(10, 5))
plt.plot(generation, stats.get_fitness_mean(), label="Mean Fitness")
plt.plot(generation, stats.get_fitness_max(), label="Max Fitness")
plt.xlabel("Generation")
plt.ylabel("Fitness")
plt.title("Fitness over Generations")
plt.legend(loc="best")
plt.show()